In [ ]:
import pandas as pd
import miceforest as mf

In [ ]:
file_path = '/finngen/library-red/EA5/proteomics/olink/third_batch/QCd/proteomics_QC_all.txt'
finngen_olink = pd.read_csv(file_path, sep='\t')

In [ ]:
# rename columns
finngen_olink = finngen_olink.rename(columns={'NT-proBNP': 'NTproBNP2', 'ERVV-1': 'ERVV_1', 'HLA-A': 'HLA_A', 'HLA-DRA': 'HLA_DRA', 'HLA-E': 'HLA_E'})

# Replace 'NTproBNP2' with the duplicate column name
finngen_olink['NTproBNP'] = finngen_olink[['NTproBNP', 'NTproBNP2']].bfill(axis=1).iloc[:, 0]
finngen_olink.drop(columns='NTproBNP2', inplace=True)  

In [ ]:
# Calculate the threshold for missing data
perc = 0.3
threshold = perc * len(finngen_olink.index)

# Count the number of missing values in each column
mperc = (finngen_olink.isnull().sum() / len(finngen_olink.index)) * 100

# Filter columns where missing data is over 30%
cols_over = mperc[mperc > (perc*100)].index

print(f"Column names with missing data over {perc*100}%: {cols_over}")

In [ ]:
# FID = finngen_olink['FID']
# finngen_olink.set_index('FID', inplace=True)
finngen_olink.drop('IID', axis=1, inplace=True)

In [ ]:
## Create a dictionary of variables use to impute

# columns that should not be used to impute missing (including 3 proteins with high missing)
exclude = ['FID'] + list(cols_over)
# variables we don't want imputed
dont_impute = ['FID']
# create dict with list of predictors for each protein
column_dict = {col: [other_col for other_col in finngen_olink.columns if other_col != col and other_col not in exclude] for col in finngen_olink.columns if col not in dont_impute}

In [ ]:
'FID' in column_dict['APOE']

In [ ]:
finngen_olink

In [ ]:
finngen_olink['FID'] = pd.Categorical(finngen_olink['FID'])
finngen_olink

In [ ]:
# set random seed
random_seed = 3456

# run miceforest imputation on multiple cores
kds = mf.ImputationKernel(
  finngen_olink,
  datasets=1,
  variable_schema=column_dict,
  random_state=random_seed
)

# run
kds.mice(
  iterations=5,
  n_jobs=-1, 
  verbose=True
)

# get the completed dataframe from the miceforest object
olink_data_imputed = kds.complete_data()

#save imputed data
olink_data_imputed.to_csv('/home/ivm/Documents/olink_imputed_jan_25_2024.csv', index=None)

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import BayesianRidge

# initialize imputer
imputer = IterativeImputer(
#     estimator=RandomForestRegressor(),
    estimator=BayesianRidge(),
    max_iter=5,
    n_nearest_features=100,
    random_state=3456,
    verbose=2
)

# fit imputer
imputed = imputer.fit_transform(finngen_olink)
finngen_olink_imputed = pd.DataFrame(imputed, columns=finngen_olink.columns, index=finngen_olink.index)


In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np

imp = SimpleImputer(
    missing_values=np.nan, 
    strategy='mean'
)

# fit imputer
imputed = imp.fit_transform(finngen_olink)
finngen_olink_imputed = pd.DataFrame(imputed, columns=finngen_olink.columns, index=finngen_olink.index)

In [ ]:
finngen_olink_imputed

In [ ]:
finngen_olink_imputed['FID'] = finngen_olink_imputed.index
finngen_olink_imputed.to_csv('/home/ivm/Documents/olink_imputed_dec_7_2023.csv', index=None)